# Inspecting the TFTI/DeepSEA data set

In [3]:
import sys
import tensorflow as tf
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Reading dummy tf-records

In [4]:
sys.path.append("../tfti")
import tfti

p = tfti.DeepseaProblem()
hparams = tfti.transformer.transformer_base_single_gpu()
hparams.chunk_size = p.default_chunk_size

filenames = tf.gfile.Glob(f"../tfti/dev/{p.name}-*")
ds = tf.data.TFRecordDataset(filenames)
ds = ds.batch(1)  # Batch size of one.
ds = ds.repeat()  # Repeat forever.
example = ds.make_one_shot_iterator().get_next()
example = tf.parse_example(example, p.example_reading_spec()[0])

sess = tf.Session()
example = sess.run(example)
# Convert uint-8 encoding to string.
inputs = "".join(map(chr, example["inputs"][0]))
targets = example["targets"][0]

## Visualizations

In [5]:
import string
import pandas as pd
import seaborn as sns
from sty import fg, bg, ef, rs

def chr_color_print(string, chrs=string.printable, l=0.8, s=0.8):
    """Prints a color string. Each character gets mapped to a unique color."""
    colors = sns.hls_palette(len(chrs), l=l, s=s)
    colors = (np.array(colors) * 255.0).astype(np.uint8)
    chr_to_color = dict(zip(chrs, colors))
    chrs = [bg(*chr_to_color[c]) + c + bg.rs for c in string]
    print("".join(chrs))

print("Inputs:")
chr_color_print(inputs, chrs="NACTG")

print("\nTargets:")
chr_color_print("".join(map(str, targets)), chrs="01")

print("\nPositive labels:")
url = "http://deepsea.princeton.edu/media/help/posproportion.txt"
df = pd.read_csv(url, delimiter="\t")
df[pd.Series(targets.astype(np.bool))]

Inputs:
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAAATTTGAAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTAATTTTAATTTTAATTTTAATTTTAATTTTAATTTGAATTTTGAATTTGAATTTGAATTTGATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTGAATTTTGAATTTGAATTTGAATTTGAATTTGTCTCCGATTTGTACTTCCTTTCTTTCTTTCCCGTGCATTGCACCACAATGCGCTGTTCTTGGTACACGATTATTCAAAGTGCGCTACACCATAATCTACTGTTCTTTGTCTCCGCTGTGTTCCC

,Data Source,Cell Type,TF/DNase/HistoneMark,Treatment,Positive Proportion
60,ENCODE,HUVEC,DNase,None,0.049407
67,ENCODE,GM12864,DNase,None,0.039357
76,ENCODE,HCF,DNase,None,0.046528
371,ENCODE,K562,ZBTB33,None,0.001554
825,Roadmap Epigenomics,H1-hESC,H3K23me2,None,0.023001
833,Roadmap Epigenomics,H1-hESC,H3K56ac,None,0.014907
837,Roadmap Epigenomics,H1-hESC,H3K9me3,None,0.025200
838,Roadmap Epigenomics,H1-hESC,H4K20me1,None,0.027315
840,Roadmap Epigenomics,H1-hESC,H4K8ac,None,0.074433
863,Roadmap Epigenomics,Monocytes-CD14+RO01746,H3K9me3,None,0.051683
